In [1]:
tweet_events = []
filenames = ['./project-data/train.data.txt', './project-data/train.label.txt']

def gen_line(filename):
    with open(filename) as f:
        for line in f:
            yield line.strip()

gens = [gen_line(n) for n in filenames]
idx = 0
for data, label in zip(*gens):
    tweet_event = {}
    ids = data.split(',')
    idx += 1
    tweet_event["index"] = idx
    tweet_event["ids"] = ','.join(ids)
    tweet_event["is_rumor"] = label
    tweet_events.append(tweet_event)
    
tweet_events

[{'index': 1,
  'ids': '1250219300389974016,1250219116993974272,1250219437027766273,1250219620939657216,1250219777185873922,1250219894429208577,1250219998842216448,1250220115762667520,1250220272306638848,1250220389323526146,1250220527005753344,1250220791544705025,1250220987238383616,1250221140603047937,1250221275827470336,1250221402822545410',
  'is_rumor': 'nonrumour'},
 {'index': 2,
  'ids': '554886875303780352,554894001946759168,554959644125167617,554927149090897920,554892798869389312,554897957372567552,554903624808398850,554895301316591616,554902157942480896,554902638220042240,554915829171515392,554965654437384192,554898706680778753,554896019419574273,554893579957460992,554899738575642624,554895363165790209,554900417071833088,554907945377726464,554893028394278912,554956172529520640,554887349180833793,554890068603969536,554892765255847936,554928464676274177,554891144480718848,554961344676392960,554919484473880576,554895244412473346,554986669527818240,554919544310202368,5549086451634

In [15]:
import json
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))

invalid_events = 0
features = dict()
features['context_annotations'] = dict()
events_data = []

for event in tweet_events:
    ids = event['ids'].split(',')
    is_rumor = event['is_rumor']
    
    tweet_objects = []
    event_list = []
    
    for tweet_id in ids:
        try:
            with open('./train_dev_data/train/' + tweet_id + '.json') as tweet_str:
                for line in tweet_str:
                    tweet = json.loads(line)
                    tweet_objects.append(tweet)
        except:
            continue
            
    if (len(tweet_objects) == 0):
        invalid_events += 1
        
    else:
        for tweet in tweet_objects:
            data = dict()
            
            # finds all the possible features contained within a tweet object with their counts
            for feat in tweet:
                if feat == 'context_annotations':
                    for feat2 in tweet[feat][0]:
                        if type(tweet[feat][0][feat2]) == dict:
                            for feat3 in tweet[feat][0][feat2]:
                                if feat in features:
                                    if feat2 in features[feat]:
                                        if feat3 in features[feat][feat2]:
                                            features[feat][feat2][feat3] += 1
                                        else:
                                            features[feat][feat2][feat3] = 1
                                    else:
                                        features[feat][feat2] = dict()
                                        features[feat][feat2][feat3] = 1
                                else:
                                    features[feat] = dict()
                                    features[feat][feat2] = dict()
                                    features[feat][feat2][feat3] = 1
                        else:
                            if feat2 in features[feat]:
                                features[feat][feat2] += 1
                            else:
                                features[feat][feat2] = 1
                elif type(tweet[feat]) == dict:
                    for feat2 in tweet[feat]:
                        if feat in features:
                            if feat2 in features[feat]:
                                features[feat][feat2] += 1
                            else:
                                features[feat][feat2] = 1
                        else:
                            features[feat] = dict()
                            features[feat][feat2] = 1
                else:
                    if feat in features:
                        features[feat] += 1
                    else:
                        features[feat] = 1
            
            # compiling features of each tweet object
            text = tweet['text']
            text_removal_list = []
            text_tokens = set(tt.tokenize(text.lower()))

            for token in text_tokens:
                # if token is a link
                if (token.startswith('https') or token.startswith('http')):
                    text_removal_list.append(token)
                
                # if token doesn't contain any letters
                elif not any(char.isalpha() for char in token):
                    text_removal_list.append(token)

                # if token is found in stopwords
                elif (token in stopwords):
                    text_removal_list.append(token)

            # remove all compiled tokens in for loop above
            for token in text_removal_list:
                text_tokens.remove(token)
            
            lang = tweet['lang']
            author_id = tweet['author_id']
            
            if 'context_annotations' in tweet:
                entity = tweet['context_annotations'][0]['entity']
                entity_id = entity['id']
                entity_name = entity['name']
                if 'description' in tweet['context_annotations'][0]['domain']:
                    author_des = tweet['context_annotations'][0]['domain']['description']
                    author_removal_list = []
                    author_des_tokens = set(tt.tokenize(author_des.lower()))

                    for token in author_des_tokens:
                        # if token is a link
                        if (token.startswith('https') or token.startswith('http')):
                            author_removal_list.append(token)

                        # if token doesn't contain any letters
                        elif not any(char.isalpha() for char in token):
                            author_removal_list.append(token)

                        # if token is found in stopwords
                        elif (token in stopwords):
                            author_removal_list.append(token)

                    # remove all compiled tokens in for loop above
                    for token in author_removal_list:
                        author_des_tokens.remove(token)
                        
                else:
                    author_des_tokens = None
            else:
                entity_id = data['entity_name'] = data['author_des'] = None
        
            if 'entities' in tweet:
                hashtags = []
                mentions = []
                annotations = []
                if 'hashtags' in tweet['entities']:
                    for hashtag in tweet['entities']['hashtags']:
                        hashtags.append(hashtag['tag'])
                if 'mentions' in tweet['entities']:
                    for mention in tweet['entities']['mentions']:
                        mentions.append(mention['id'])
                if 'annotations' in tweet['entities']:
                    for annotation in tweet['entities']['annotations']:
                        annotations.append((annotation['normalized_text'], annotation['type'], annotation['probability']))
                        
            # compiling data
            data = [text_tokens, lang, author_id, entity_id, entity_name, author_des_tokens, hashtags, mentions, annotations]
            event_list.append(data)
            
    events_data.append((event_list, is_rumor))    

{'regularly', 'saline', 'help', 'coronavirus', 'new', 'infection', 'prevent', 'nose', 'rinsing'}
{'help', 'coronavirus', '#covid19malaysia', 'new', 'infection', 'garlic', 'eating', 'prevent'}
{'protect', 'coronavirus', 'new', 'vaccines', 'pneumonia'}
{'body', '#chamber', 'alcohol', 'coronavirus', 'kill', 'new', 'chlorine', 'spraying'}
{'detecting', 'effective', 'coronavirus', 'scanners', 'new', 'people', 'infected', 'thermal'}
{'coronavirus', 'disinfection', 'kill', 'new', 'lamp', 'ultraviolet'}
{'effective', 'coronavirus', 'killing', 'new', 'dryers', 'hand'}
{'coronavirus', 'cannot', 'new', 'transmitted', 'mosquito', 'bites'}
{'taking', 'coronavirus', 'disease', 'new', 'prevent', 'bath', 'hot'}
{'cannot', 'coronavirus', 'kill', 'snow', 'new', 'cold', 'weather'}
{'humid', 'climates', 'covid', 'virus', 'areas', 'transmitted', 'hot'}
{'protect', 'alcohol', 'covid', 'dangerous', 'drinking'}
{'covid', 'feeling', 'mean', 'lung', 'seconds', 'hold', 'without', 'breath', 'free', 'able', 'cough

{'detecting', 'effective', 'kill', 'infected', 'dryers', 'scanners', 'lamp', 'people', 'disinfection', 'ultraviolet', 'hand', 'coronavirus', 'killing', 'new', 'thermal'}
{'effective', 'prevent', 'younger', 'also', 'preventing', 'help', 'susceptible', 'affect', 'people', 'older', 'coronavirus', 'new', 'infection', 'treating', 'garlic', 'eating', 'antibiotics'}
{'mowing', 'yards', "he's", '@urbancraziness', 'hoptown', '@boren_madison'}
{'cant', 'believe', 'still', 'tupac', '@urbancraziness', 'alive'}
{'cant', 'believe', 'still', 'tupac', '@urbancraziness', '@dickwhiskeyy', 'alive'}
{'cant', 'believe', '@maui_wowwie', 'still', 'tupac', '@urbancraziness', 'alive'}
{'cant', 'believe', 'still', 'tupac', '@urbancraziness', 'alive'}
{'cant', 'believe', "y'all", 'still', 'tupac', '@urbancraziness', 'alive', 'ayyeee'}
{'cant', 'believe', 'still', 'tupac', '@urbancraziness', 'alive', '@kingdoig21'}
{'cant', '@thechomason', 'believe', 'still', 'tupac', '@urbancraziness', 'alive'}
{'cant', 'believe

{'nairobi', 'covid', 'sonko', 'warns', 'giving', 'packs', 'mike', 'note', 'people', 'small', 'governor', 'drinking', 'protect', 'alcohol', 'bottles', 'hennessy', 'food', 'dangerous'}
{'like', 'looks', 'band', 'top', 'blue', 'plastic', '@citizentvkenya', 'guy', 'bottle'}
{'effective', 'holy', 'taken', 'orally', 'sins', 'saints', "what's", 'wondering', 'sanitizer', 'even', 'microorganism', 'body', 'form', '@citizentvkenya', 'hennessy', 'cleanses', 'born'}
{'@jibrilkala', 'body', 'helping', 'alcohol', '@citizentvkenya', 'immunity', 'reduces'}
{'need', 'sonko', 'gov', 'mike', '@mikesonko', 'cheering', 'honest', 'thanks', '#henneybottlechallenge', '#hennessy', '@citizentvkenya', 'coronavirus', 'times', 'bit', 'stressful'}
{'jamaa', '@kelvin__mungai', 'kila', 'ako', 'ehhh', 'vituko', 'time', 'na', '@citizentvkenya'}
{'spirit', '@citizentvkenya', "that's"}
{'nice', '@_tweeted_', 'one', '@citizentvkenya'}
{'akili', 'mtu', 'sonko', 'think', 'yake', 'venye', 'alafu', 'ya', 'kuvote', 'ni', 'mara'

{'@drudge_report', 'know', 'talking', 'even', 'jendal', "i'm", "that's"}
{'@drudge_report', 'good', 'luck', 'getting'}
{'tell', 'louisiana', '@drudge_report', 'meet', 'stop', 'entering', 'jindal'}
{'agree', 'totally', 'wow', '@drudge_report', 'jindal'}
{'@drudge_report', '#returntosender', 'refugees'}
{'damn', 'jindal', 'pretty', 'problem', 'national', '@drudge_report', '#tcot', 'demands', 'call', 'information', 'stop', '@ellerich17', 'guard', 'late'}
{'said', 'jindal', '@drudge_report', 'weak', '@bobbyjindal', 'state', 'sender', 'return'}
{'ever', 'refugees', '@drudge_report', 'syrian', '@bobbyjindal', 'needs', 'immigration', 'anything', 'life', 'louisiana', 'stop'}
{'@drudge_report', 'stop', 'power'}
{'states', 'everyone', 'ther', 'benefit', '@drudge_report', 'citizens', '@traderstef', 'united'}
{'send', 'demand', 'homr', 'amendment', '@drudge_report', 'back', 'info', 'effing', 'round', 'sent', '10th'}
{'wow', '@drudge_report', '@love_chihuahua'}
{'memphis', '@drudge_report', 'come',

{'remember', 'step', 'correction', '@priyapyadav18', 'without', 'always', 'little', 'hm', 'dm', 'pm', 'fm', "can't", 'direction'}
{'@priyapyadav18', '@nijam35214500'}
{'@priyapyadav18', 'charity', 'aam', 'admi', 'first', '@bjp4india', 'empty', 'treasury', 'ask', 'begins', 'home'}
{'@priyapyadav18', 'morning', 'well', 'good', 'priya', 'beta'}
{'steal', '@priyapyadav18', 'ways', 'new', 'swindle', 'money', 'public'}
{'privately', '@priyapyadav18', 'citizens', 'private', 'funds', 'govt', 'money', 'use', 'get'}
{'go', 'shame', 'watching', 'pockets', 'karma', 'paisa', 'help', 'god', 'jayenge', '@priyapyadav18', 'sara', 'crores', 'hope', 'lords', 'kha', 'taking', 'pouring', 'action', 'many'}
{'sagging', 'image', '@priyapyadav18', 'waking', 'retrieve', 'end', 'public', 'beginning'}
{'sham', '@priyapyadav18', 'maharashtra', 'scam', 'government', 'collecting', 'complete', 'topple'}
{'good', '@priyapyadav18'}
{'@priyapyadav18', 'another', 'little', 'scam'}
{'away', 'gets', 'think', 'moron', '@pri

...

{'whole', 'covid', 'go', 'map', 'pick', 'work', 'live', 'contaminated', 'come', 'people', '@noltenc', 'back', 'cases', '@jkc_jaycee', 'showing', 'say', "there's", 'want', 'get', 'corridor', 'care', 'air', '@madsquire', 'ppl', 'sick'}
{'comments', 'covid', 'ask', 'think', 'statement', 'sure', 'contaminated', 'whether', 'concerns', '@noltenc', 'smart-ass', 'nothing', '@jkc_jaycee', 'say', 'trying', 'deep', 'realize', "i'm", 'dig', '@madsquire', 'know', 'made'}
{'whole', 'covid', 'go', 'map', 'pick', 'work', 'live', 'contaminated', 'come', 'people', '@noltenc', 'back', 'cases', '@jkc_jaycee', 'showing', 'say', "there's", 'want', 'get', 'corridor', 'care', 'air', '@madsquire', 'ppl', 'sick'}
{'low', '@jrharrison1', 'scared', 'show', '@noltenc', 'may', 'wish', 'nothing', '@jkc_jaycee', 'fear', 'unless', 'recent', 'antibody', 'afraid', 'tests', 'likely', 'rate', 'since', 'stop', 'know', 'fatality', 'late', 'wanting', 'choose'}
{'got', 'station', 'going', 'live', "think's", 'community', 'come

{'thank', 'happy', 'wud', 'atlast', 'b', '@bbcbreaking', 'parent', 'god'}
{'@bbcbreaking', 'boko', 'abducted', 'says', 'agreed', "nigeria's", 'truce', 'w', 'haram', 'released', 'military', 'finally', 'schoolgirls', 'mt'}
{'anxiety', 'lol', 'power', 'diplomatic', 'regime', 'seems', '@bbcbreaking', 'full', 'truce', 'iam'}
{'wait', 'well', '@mapetiteniche'}
{'abducted', 'says', 'agreed', 'haram', 'schoolgirls', "nigeria's", 'released', 'military', 'truce', 'boko', 'group', 'militant'}
{'hear', 'gud', '@bbcbreaking'}
{'believe', '@bbcnews', 'well', '@bbcbreaking', 'girls', 'alive', 'see', "i'll"}
{'ceasefire', 'includes', '@bbcbreaking', 'release', 'hostages', 'boko', 'says', 'haram', 'nigerian', '#bringbackourgirls', 'government'}
{'@bbcbreaking'}
{'@bbcbreaking', 'ilks', 'believing', 'trusted', '#bokoharam', 'dream', "can't", '#bringbackourgirls', 'seeing'}
{'@bbcbreaking', 'good', 'almost', 'true'}
{'@bbcbreaking'}
{'ceasefire', 'includes', '@bbcbreaking', 'release', 'hostages', 'boko',

{'another', 'pt', 'national', '@aparanjape', 'training', 'teacher', 'skill', 'trainer', 'physical', 'education', 'document', 'activity', 'pe', 'sports', 'corporation', 'development', 'name'}
{'@prayagverma', 'yes', 'course', 'role'}
{'skills', 'outside', 'lack', 'acchhe', 'us', 'k12', '@aparanjape', 'ai', 'space', 'support', 'schools', 'din', 'hoping', 'curriculum', 'look', 'dt'}
{'wud', 'paper', 'expecting', 'u', 'post-it', '@aparanjape', 'notes', 'skill', 'studied', 'mba', 'hardware', 'build', 'drawing', 'much', 'using', 'something', 'able', 'end', 'except', 'teach', 'design', 'software', 'talk', 'thinking', 'learn', 'technical'}
{'person', '@aparanjape', 'right', 'taught', 'journeys', '@vedic_guru', 'probably', 'prototyping', 'critical', 'used', 'user', 'aspects', 'design', 'thinking', 'tools'}
{'drill', 'master', '@aparanjape', 'cha', 'p'}
{'much', 'fifty', 'leaving', 'yeah', 'jamie', 'grey', 'dornan', 'shades'}
{'leaving', 'invested', 'well', 'shades', 'yeah', 'much', 'fifty', "i'

{'kill', 'busters', 'spraying', 'q', '#covid19', 'viruses', 'body', 'alcohol', 'chlorine', 'already', 'entered', '#coronavirus', 'myth', 'new'}
{'detect', 'detecting', 'effective', 'infected', 'busters', 'q', 'scanners', 'fever', 'people', 'however', "can't", '#coronavirus', 'yet', 'myth', 'new', 'infection', 'sick', 'developed', 'thermal'}
{'effective', 'bacteria', 'preventing', 'busters', 'work', 'q', 'virus', 'treatment', 'viruses', 'ncov', 'therefore', 'used', '#covidー19', '#coronavirus', 'myth', 'new', 'treating', 'antibiotics', 'means', 'prevention'}
{'asthma', 'ages', 'like', 'infected', 'busters', 'younger', 'also', 'susceptible', 'q', '#covid19', 'affect', 'med', 'people', 'vulnerable', 'pre-existing', 'appear', 'older', 'diabetes', 'conditions', '#coronavirus', 'coronavirus', 'myth', 'new'}
{'person', 'saliva', 'mosquito', 'spread', 'infected', 'respiratory', '#covid19', 'virus', 'transmitted', 'discharge', 'droplets', 'sneezes', 'nose', 'bites', 'coughs', '#coronavirus', 'ca

{'hib', 'protection', 'protected', 'b', 'saline', '#corona', 'people', 'nose', 'type', 'provide', 'pneumonia', 'rinsing', 'regularly', 'haemophilus', 'vaccine', 'coronavirus', 'influenza', 'new', 'infection', 'evidence', 'vaccines', 'pneumococcal'}
{'protected', 'outbreak', 'antimicrobial', '#corona', 'properties', 'people', 'may', 'however', 'healthy', 'current', 'coronavirus', 'new', 'garlic', 'food', 'evidence', 'eating'}
{'asthma', 'ages', 'severely', 'infected', 'ill', 'medical', 'virus', 'people', 'vulnerable', 'becoming', 'pre-existing', 'ncov', 'appear', 'older', 'heart', 'diabetes', 'conditions', 'coronavirus', 'disease', 'new'}
{'recommended', 'treat', 'date', 'bacteria', 'prevent', 'work', '#corona', 'viruses', 'ncov', 'coronavirus', 'medicine', 'new', 'specific', 'antibiotics'}
{'desperate', 'shooter', 'leftis', 'read', 'transgender', 'planned', 'rumor', 'ted', 'based', 'claims', 'parenthood', 'cruz'}
{'narrative', 'issue', '@lacina52', 'real', 'obviously', 'pushing', '@bip

In [9]:
# features and their counts in tweet_events from commented block above (total of 21775 tweet objects)
features

{'context_annotations': {'domain': {'id': 4578,
   'name': 4578,
   'description': 4576},
  'entity': {'id': 4578, 'name': 4578, 'description': 1386}},
 'conversation_id': 21775,
 'author_id': 21775,
 'attachments': {'media_keys': 4100, 'poll_ids': 20},
 'in_reply_to_user_id': 20455,
 'entities': {'urls': 6284,
  'hashtags': 3401,
  'mentions': 17722,
  'annotations': 6656,
  'cashtags': 15},
 'id': 21775,
 'created_at': 21775,
 'possibly_sensitive': 21775,
 'lang': 21775,
 'reply_settings': 21775,
 'text': 21775,
 'source': 21765,
 'referenced_tweets': 20496,
 'public_metrics': {'retweet_count': 21775,
  'reply_count': 21775,
  'like_count': 21775,
  'quote_count': 21775},
 'geo': {'coordinates': 395, 'place_id': 869},
 'withheld': {'copyright': 14, 'country_codes': 14}}

In [4]:
import torch